In [9]:
import re
from pymongo import MongoClient 
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import jieba
jieba.set_dictionary('D:/text_mining/dict.txt')  #切換至中文繁體字庫
jieba.load_userdict('D:/text_mining/dict_keyw_new.txt')  #
jieba.load_userdict('D:/text_mining/dict_cbdic.txt')  #
#預設就是自己
client = MongoClient('10.1.21.204',27017)  #不用修改  字典檔請改位子
#client = MongoClient('localhost:27017')  #不用修改  字典檔請改位子

database = client['test']
collection =database['test']

Building prefix dict from D:\text_mining\dict.txt ...
Loading model from cache c:\users\ricky.pan\appdata\local\temp\jieba.u7d52069ae8c3dad20cb6bf6947fd3df2.cache
Loading model cost 3.853 seconds.
Prefix dict has been built succesfully.


### 執行前修改comp:報社 date:修改年份(下方共有3個find需修改)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import time

for ii in range (3,4):   # 1-9月    10月後改為10-13再執行
    a=collection.find(
        {"$and":[                   
                {"comp":{"$regex":"apple"}},
                {"date":{"$regex":"20160{}".format(ii)}}           #計算當月文章數
                ]},{"_id":0}).count()    
    content =[]
    tStart=time.time() #計算所需時間(計時開始)
    
    for post in collection.find(
        {"$and":[                   
                {"comp":{"$regex":"apple"}},                              #尋找一個月新聞(報社手動修改)
                {"date":{"$regex":"20160{}".format(ii)}}        #10-12月請將0去掉
                ]}): 
        summary = post['content']
        content.append('/'.join(jieba.cut(summary)))

    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(content)  # titile 放文本
    weight = X.toarray()
    features = vectorizer.get_feature_names()     # 拿到所有的關鍵詞  

    top_features = []
    for n in range(0,a):  #迴圈參考上面的總文章數
        indices = np.argsort(weight[n])[::-1]  # transformer = TfidfTransformer()  #X.toarray()[5] 是第幾篇新聞的意思
        # 看TOP多少的詞
        top_n = 40
        top_features.append([features[i] for i in indices[:top_n]])    #這邊的寫法會讓關鍵字中間會有空白     

    obj=collection.find(
        {"$and":[                                                                  #尋找一個月新聞(報社手動修改)
                {"comp":{"$regex":"apple"}},                               
                {"date":{"$regex":"20160{}".format(ii)}}
                ]})
    for i in range(0,a):  #迴圈參考上面的總文章數
        collection.update({"_id":obj[i]["_id"]},{"$set":{"tfidf":top_features[i]}}) #collection.update即可

    tEnd = time.time() #記時(時間結束)
    print '完成',len(top_features),'筆'    
    
    
    
    del content[:]  #清除list暫存記憶體
    del top_features[:]  #清除list暫存記憶體
    
    print ' 花了 {} s'.format(tEnd-tStart)    #將經過時間print 出來

MemoryError: 